In [1]:
import chess.pgn
import pandas as pd
import numpy as np

Посчитаем, сколько записей в нашей дате подходят под наши нужды

In [2]:
i = 0
required_headers = ['Event', 'Result', 'WhiteElo', 'BlackElo', 'WhiteRatingDiff', 'BlackRatingDiff', 'ECO', 'Opening',
                    'TimeControl', 'Termination']
with open("../data/lichess_db_standard_rated_2013-01.pgn") as pgn:
    while True:
        game = chess.pgn.read_game(pgn)
        if game is None:
            break

        if not all(header in game.headers for header in required_headers) or '?' in game.headers['Event'] + \
                game.headers['Result'] + game.headers['WhiteElo'] + game.headers['BlackElo'] + game.headers[
            'WhiteRatingDiff'] + game.headers['BlackRatingDiff'] + game.headers['ECO'] + game.headers[
            'Opening'] + game.headers['TimeControl'] + game.headers['Termination']:
            continue
        i += 1
print(i)

121114


Давайте теперь для удобства запишем данные в csv формат, чтобы в дальнейшем сильно ускорить работу с датафреймом

In [3]:
N = i

events = np.empty(N, dtype=object)
results = np.empty(N, dtype=object)

white_elo = np.empty(N, dtype=int)
black_elo = np.empty(N, dtype=int)
white_rating_diff = np.empty(N, dtype=int)
black_rating_diff = np.empty(N, dtype=int)

ecos = np.empty(N, dtype=object)
openings = np.empty(N, dtype=object)

time_control = np.empty(N, dtype=object)
termination = np.empty(N, dtype=object)

all_moves = np.empty(N, dtype=object)

i = 0
with open("../data/lichess_db_standard_rated_2013-01.pgn") as pgn:
    while True:
        game = chess.pgn.read_game(pgn)
        if game is None:
            break
            
        if not all(header in game.headers for header in required_headers) or '?' in game.headers['Event'] + \
                game.headers['Result'] + game.headers['WhiteElo'] + game.headers['BlackElo'] + game.headers[
            'WhiteRatingDiff'] + game.headers['BlackRatingDiff'] + game.headers['ECO'] + game.headers[
            'Opening'] + game.headers['TimeControl'] + game.headers['Termination']:
            continue

        moves = []
        for item in game.mainline_moves():
            moves.append(str(item))
            
        events[i] = game.headers['Event']
        results[i] = game.headers['Result']
        white_elo[i] = game.headers['WhiteElo']
        black_elo[i] = game.headers['BlackElo']
        white_rating_diff[i] = game.headers['WhiteRatingDiff']
        black_rating_diff[i] = game.headers['BlackRatingDiff']
        ecos[i] = game.headers['ECO']
        openings[i] = game.headers['Opening']
        time_control[i] = game.headers['TimeControl']
        termination[i] = game.headers['Termination']
        all_moves[i] = moves

        i += 1

df = pd.DataFrame({'Events': events, 'results': results, 'white_elo': white_elo, 'black_elo': black_elo,
                   'white_rating_diff': white_rating_diff, 'black_rating_diff': black_rating_diff, 'ecos': ecos,
                   'openings': openings, 'time_control': time_control, 'termination': termination,
                   'all_moves': all_moves})

Проверим, что получили желаемый результат

In [4]:
df.head()

,Events,results,white_elo,black_elo,white_rating_diff,black_rating_diff,ecos,openings,time_control,termination,all_moves
0,Rated Classical game,1-0,1639,1403,5,-8,C00,French Defense: Normal Variation,600+8,Normal,"[e2e4, e7e6, d2d4, b7b6, a2a3, c8b7, b1c3, g8h..."
1,Rated Classical game,1-0,1654,1919,19,-22,D04,"Queen's Pawn Game: Colle System, Anti-Colle",480+2,Normal,"[d2d4, d7d5, g1f3, g8f6, e2e3, c8f5, f3h4, f5g..."
2,Rated Classical game,1-0,1643,1747,13,-94,C50,Four Knights Game: Italian Variation,420+17,Normal,"[e2e4, e7e5, g1f3, b8c6, f1c4, g8f6, b1c3, f8c..."
3,Rated Bullet game,0-1,1824,1973,-6,8,B12,Caro-Kann Defense: Goldman Variation,60+1,Normal,"[e2e4, c7c6, b1c3, d7d5, d1f3, d5e4, c3e4, b8d..."
4,Rated Bullet game,0-1,1765,1815,-9,9,C00,French Defense: La Bourdonnais Variation,60+1,Normal,"[e2e4, e7e6, f2f4, d7d5, e4e5, c7c5, g1f3, d8b..."


Запишем в нужную папку получившуюся дату

In [5]:
df.to_csv("../data/lichess_db_standard_rated_2013-01.csv", encoding='utf-8', index=False)